In [28]:
import sys
sys.path.append("../")
import consts
import api
from subset import addr_operation
from Anonymizer import anonymizer
import pickle
from copy import deepcopy

attr_list = consts.ATTR_LIST
attr_list.append('seq')

original_file = '../evaluation/original_data.csv'
anonymized_file = '../' + consts.ORIGIN_FILE
watermarked_file = '../' + consts.MODIFIED_FILE

csv_header, org_list = api.parsed_list(original_file, True)
_, anonymized_list = api.parsed_list(anonymized_file, True)
_, watermarked_list = api.parsed_list(watermarked_file, True)

In [32]:
def calc_addr_tree(addr_tree, addr, addr_attr):
    for addr_freq in addr_tree:
        if addr == addr_freq[0]:
            addr_freq[1] += 1
            break
    else:
        addr_tree.append([addr, 1])
    if addr == ['関東', '*', '*', '*', '*']:
        return
    else:
        calc_addr_tree(addr_tree, anonymizer.address_masking(addr_attr, deepcopy(addr)), addr_attr)

In [31]:
def addr_tree(addr_l):
    # treeじゃないけど
    # addr_tree = [addr_freq, ...]
    # addr_freq = [[listed_generalized_addr], freq]
    addr_attr = ['addr0', 'addr1', 'addr2', 'addr3', 'addr4']
    
    addr_tree = list()
    
    for addr in addr_l:
        calc_addr_tree(addr_tree, addr, addr_attr)
    return addr_tree

In [4]:
def loss(attr_type, tree, bottom_attr, top_attr=None):
    if top_attr is not None:
        return
    return

In [35]:
def IL_calc(org_l, mod_l, attr_list):
    # sequential numberをintに
    for org_r in org_l:
        org_r.append(int(org_r.pop()))
    for mod_r in mod_l:
        mod_r.append(int(mod_r.pop()))
    
    # sort by sequential number
    org_l.sort(key=lambda x: x[-1])
    mod_l.sort(key=lambda x: x[-1])
    
    # insert empty record to mod_l
    for i, mod_r in enumerate(mod_l):
        if i != mod_r[-1]:
            mod_l.insert(i, ['empty', i])
    
    # IL of addr
    addr_first, addr_last = addr_operation.addr_range_catcher(attr_list)
    
    ## get original addr list
    org_addr_l = [x[addr_first:addr_last+1] for x in org_l]
    
    ## get modified addr list
    mod_addr_l = list()
    for mod_r in mod_l:
        if mod_r[0] == 'empty':
            mod_addr_l.append(['関東', '*', '*', '*', '*'])
        else:
            mod_addr_l.append(mod_r[addr_first:addr_last+1])
        
    ## address tree from original addr list
    addr_tree_ = addr_tree(org_addr_l)
    
    ## IL calculation
    '''
    for i, mod_addr in enumerate(mod_addr_l):
        org_addr = org_addr_l[i]
        numerator = loss('addr', addr_tree_, org_addr)
        denominator = numerator + loss('addr', addr_tree_, mod_addr, org_addr)
        IL = numerator / denominator
    '''
    
    return

In [36]:
IL_calc(org_list, anonymized_list, attr_list)